In [18]:
import os
import json

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern

if os.getcwd().endswith('notebook'):
    os.chdir('..')

from rna_learn.hyperparameters import expected_improvement, propose_location

In [2]:
sns.set(palette='colorblind', font_scale=1.3)

In [3]:
path_to_trace = os.path.join(os.getcwd(), 'hyperparameters/run_1eo28/trace.json')

In [4]:
m52 = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5)
m52_s = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5)

gpr = GaussianProcessRegressor(kernel=m52)
gpr_s = GaussianProcessRegressor(kernel=m52_s)

In [5]:
with open(path_to_trace) as f:
    trace = json.load(f)['trace']

In [6]:
trace[0]

{'x': {'n_epochs': 18,
  'batch_size': 12,
  'learning_rate': 0.02317854498433085,
  'adam_epsilon': 0.125095546820125,
  'n_conv_1': 2,
  'n_filters_1': 47,
  'kernel_size_1': 79,
  'l2_reg_1': 0.04730964904410152,
  'n_conv_2': 10,
  'n_filters_2': 54,
  'kernel_size_2': 41,
  'l2_reg_2': 0.07996356530213358,
  'dropout': 0.3525910668913288},
 'expected_improvement_per_second': -0.021879228006754743,
 'loss': 0.3385135233402252,
 'elapsed': 3877.710918188095}

In [10]:
params = [
    'n_epochs',
    'batch_size',
    'learning_rate',
    'adam_epsilon',
    'n_conv_1',
    'n_filters_1',
    'kernel_size_1',
    'l2_reg_1',
    'n_conv_2',
    'n_filters_2',
    'kernel_size_2',
    'l2_reg_2',
    'dropout',
]

In [7]:
X = np.array([
    [
        t['x']['n_epochs'],
        t['x']['batch_size'],
        t['x']['learning_rate'],
        t['x']['adam_epsilon'],
        t['x']['n_conv_1'],
        t['x']['n_filters_1'],
        t['x']['kernel_size_1'],
        t['x']['l2_reg_1'],
        t['x']['n_conv_2'],
        t['x']['n_filters_2'],
        t['x']['kernel_size_2'],
        t['x']['l2_reg_2'],
        t['x']['dropout'],
    ]
    for i, t in enumerate(trace)
], dtype='float32')
Y = np.array([t['loss'] for t in trace])[:, np.newaxis]
T = np.array([np.log(t['elapsed']) for t in trace])[:, np.newaxis]

In [8]:
gpr.fit(X, Y);
gpr_s.fit(X, T);

In [11]:
X_init_def = [
    dict(
        n_epochs=5,
        batch_size=32,
        learning_rate=-4,
        adam_epsilon=-7,
        n_conv_1=2,
        n_filters_1=100, 
        kernel_size_1=10,
        l2_reg_1=0.,
        n_conv_2=2,
        n_filters_2=100, 
        kernel_size_2=10,
        l2_reg_2=0.,
        dropout=0.5,
    ),
]
X_new = np.array([
    [x[p] for p in params]
    for x in X_init_def
], dtype='float32')
X_new.shape

(1, 13)

In [38]:
def expected_improvement_(X, X_sample, Y_sample, gpr, gpr_s, xi=0.01, per_second=False):
    mu, sigma = gpr.predict(X, return_std=True)
    expected_duration_ln = gpr_s.predict(X)
    expected_duration = np.exp(expected_duration_ln)
    
    sigma = sigma[..., np.newaxis]

    mu_sample_opt = np.max(Y_sample)

    with np.errstate(divide='ignore'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    if per_second:
        return ei / expected_duration
    else:
        return ei

In [39]:
expected_improvement_(X_new, X, Y, gpr, gpr_s)

(1,)
(1, 1)


array([[0.0241454]])

In [37]:
expected_improvement_(X_new, X, Y, gpr, gpr_s, per_second=True)

array([[0.0241454]])